1.異なった解の数でオラクルを作成することができますか？量子数え上げアルゴリズムの精度はどのように変化しますか?

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import math

# importing Qiskit
import qiskit
from qiskit import QuantumCircuit, transpile, assemble, Aer

# import basic plot tools
from qiskit.visualization import plot_histogram

In [5]:
def qft(n):
    #n量子ビットのQFT回路を作成
    """Creates an n-qubit QFT circuit"""
    circuit = QuantumCircuit(4)
    def swap_registers(circuit, n):
        for qubit in range(n//2):
            circuit.swap(qubit, n-qubit-1)
        return circuit
    def qft_rotations(circuit, n):
        #swapゲートなしで回路で最初にn量子ビットのQFTを実装する
        """Performs qft on the first n qubits in circuit (without swaps)"""
        if n == 0:
            return circuit
        n -= 1
        circuit.h(n)
        for qubit in range(n):
            circuit.cp(np.pi/2**(n-qubit), qubit, n)
        qft_rotations(circuit, n)
    
    qft_rotations(circuit, n)
    swap_registers(circuit, n)
    return circuit

異なる解のオラクルを作成

In [6]:
def example_grover_iteration1():
    """16状態のうち8個の解"""
    # 回路を作る
    qc1 = QuantumCircuit(4)

    qc1.h(0)
    qc1.x(0)
    qc1.h(0)
    # 拡散
    qc1.h(range(3))
    qc1.x(range(3))
    qc1.z(3)
    qc1.mct([0,1,2],3)
    qc1.x(range(3))
    qc1.h(range(3))
    qc1.z(3)
    return qc1
#回路を表示する
#回路組む時の思考を書いてあげる

In [7]:
def example_grover_iteration2():
    """16状態のうち2個の解"""
    # 回路を作る
    qc2 = QuantumCircuit(4)
 
    qc2.h(2)
    qc2.ccx(0,1,2)
    qc2.h(2)
    # 拡散
    qc2.h(range(3))
    qc2.x(range(3))
    qc2.z(3)
    qc2.mct([0,1,2],3)
    qc2.x(range(3))
    qc2.h(range(3))
    qc2.z(3)
    return qc2

In [8]:
def example_grover_iteration3():
    """16状態のうち0個の解"""
    # 回路を作る
    qc3 = QuantumCircuit(4)

    qc3.h(range(3))
    qc3.x(range(3))
    qc3.z(3)
    qc3.mct([0,1,2],3)
    qc3.x(range(3))
    qc3.h(range(3))
    qc3.z(3)
    return qc3

In [9]:
def example_grover_iteration4():
    """16状態のうち16個の解"""
    # 回路を作る
    qc4 = QuantumCircuit(4)

    qc4.h(0)
    qc4.x(0)
    qc4.h(0)
    qc4.x(0)
    qc4.h(0)
    qc4.x(0)
    qc4.h(0)
    # 拡散
    qc4.h(range(3))
    qc4.x(range(3))
    qc4.z(3)
    qc4.mct([0,1,2],3)
    qc4.x(range(3))
    qc4.h(range(3))
    qc4.z(3)
    return qc4

In [10]:
grit1 = example_grover_iteration1().to_gate()
example_grover_iteration1().draw()
grit1.label = "Grover"
cgrit1 = grit1.control()

grit2 = example_grover_iteration2().to_gate()
grit2.label = "Grover"
cgrit2 = grit2.control()

grit3 = example_grover_iteration3().to_gate()
grit3.label = "Grover"
cgrit3 = grit3.control()

grit4 = example_grover_iteration4().to_gate()
grit4.label = "Grover"
cgrit4 = grit4.control()

In [11]:
def countAnswer(cgrit):
    qft_dagger = qft(4).to_gate().inverse()
    qft_dagger.label = "QFT†"
    # 量子回路の作成
    t = 4   # カウントする量子ビットの数
    n = 4   # 探索する量子ビットの数
    qc = QuantumCircuit(n+t, t) # 古典ビットとn+t量子ビットの回路

    # 全ての量子ビットを |+>に初期化する
    for qubit in range(t+n):
        qc.h(qubit)

    # Grover反復を開始
    iterations = 1
    for qubit in range(t):
        for i in range(iterations):
            qc.append(cgrit, [qubit] + [*range(t, n+t)])
        iterations *= 2
    
    # 逆量子フーリエ変換
    qc.append(qft_dagger, range(t))

    # 測定
    qc.measure(range(t), range(t))

    aer_sim = Aer.get_backend('aer_simulator')
    transpiled_qc = transpile(qc, aer_sim)
    qobj = assemble(transpiled_qc)
    job = aer_sim.run(qobj)
    hist = job.result().get_counts()

    measured_str = max(hist, key=hist.get)
    measured_int = int(measured_str,2)
    print("Register Output = %i" % measured_int)
    theta = (measured_int/(2**t))*math.pi*2
    print("θ = %.5f" % theta)
    N = 2**n
    M = N * (math.sin(theta/2)**2)
    print("解の数 = %.1f" % (N-M))
    m = t - 1
    err = (math.sqrt(2*M*N) + N/(2**(m+1)))*(2**(-m))
    print("Error < %.2f" % err)

解が8個の場合<br><br>
xゲートをアダマールゲートで挟むことで、指定した量子ビットが1の場合、-がつく。

In [12]:
#オラクルを作成する回路を描画
qc = QuantumCircuit(4)

qc.h(0)
qc.x(0)
qc.h(0)

qc.draw()

┌───┐┌───┐┌───┐
q_0: ┤ H ├┤ X ├┤ H ├
     └───┘└───┘└───┘
q_1: ───────────────
                    
q_2: ───────────────
                    
q_3: ───────────────

In [13]:
countAnswer(cgrit1)

Register Output = 4
θ = 1.57080
解の数 = 8.0
Error < 2.12


解が2個の場合<br><br>
ccxゲートをアダマールゲートで挟むことで指定した3つの量子ビットが1の場合、-がつく

In [14]:
#オラクルを作成する回路を描画
qc2 = QuantumCircuit(4)
 
qc2.h(2)
qc2.ccx(0,1,2)
qc2.h(2)

qc2.draw()

q_0: ───────■───────
            │       
q_1: ───────■───────
     ┌───┐┌─┴─┐┌───┐
q_2: ┤ H ├┤ X ├┤ H ├
     └───┘└───┘└───┘
q_3: ───────────────

In [15]:
countAnswer(cgrit2)

Register Output = 10
θ = 3.92699
解の数 = 2.3
Error < 2.74


解が0個の場合

In [16]:
countAnswer(cgrit3)

Register Output = 8
θ = 3.14159
解の数 = 0.0
Error < 2.95


解が16個の場合<br><br>
xゲートをアダマールゲートで挟むことで、指定した量子ビットが1の場合、-がつく。<br>
また、xゲート後、xゲートをアダマールゲートで挟むことで、指定した量子ビットが0の場合にも、-をつける。

In [19]:
#オラクルを作成する回路を描画
qc4 = QuantumCircuit(4)

qc4.h(0)
qc4.x(0)
qc4.h(0)
qc4.x(0)
qc4.h(0)
qc4.x(0)
qc4.h(0)

qc4.draw()

┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐
q_0: ┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├
     └───┘└───┘└───┘└───┘└───┘└───┘└───┘
q_1: ───────────────────────────────────
                                        
q_2: ───────────────────────────────────
                                        
q_3: ───────────────────────────────────

In [20]:
countAnswer(cgrit4)

Register Output = 0
θ = 0.00000
解の数 = 16.0
Error < 0.12


A.異なる数の解のオラクルを作成することはできる。<br>
　解の数が少なくなるほど精度は低くなり、解の数が多くなるほど精度は高くなる。

2.回路を調整して、カウント量子ビットを増減して、結果の精度を変えることはできますか？

解の数の精度、エラー値はカウント量子ビットの数に依存するため、<br>
増加させると精度が高くなり、減少させると精度が低くなると思われる。

テキストと同じ解が5個のオラクルを作成。

In [100]:
def example_grover_iteration():
    """Small circuit with 5/16 solutions"""
    # 回路を作る
    qc = QuantumCircuit(4)
    # オラクル
    qc.h([2,3])
    qc.ccx(0,1,2)
    qc.h(2)
    qc.x(2)
    qc.ccx(0,2,3)
    qc.x(2)
    qc.h(3)
    qc.x([1,3])
    qc.h(2)
    qc.mct([0,1,3],2)
    qc.x([1,3])
    qc.h(2)
    # 拡散
    qc.h(range(3))
    qc.x(range(3))
    qc.z(3)
    qc.mct([0,1,2],3)
    qc.x(range(3))
    qc.h(range(3))
    qc.z(3)
    return qc

In [101]:
grit = example_grover_iteration().to_gate()
grit.label = "Grover"
example_grover_iteration().draw()
cgrit = grit.control()
#ここでグローバル変数を定義し、カウント量子ビットを調整
#countBit = 10
countBit = 3

In [102]:
def qft(n):
    #n量子ビットのQFT回路を作成
    """Creates an n-qubit QFT circuit"""
    # QuantumCircuitの引数にcountBit
    circuit = QuantumCircuit(countBit)
    def swap_registers(circuit, n):
        for qubit in range(n//2):
            circuit.swap(qubit, n-qubit-1)
        return circuit
    def qft_rotations(circuit, n):
        #swapゲートなしで回路で最初にn量子ビットのQFTを実装する
        """Performs qft on the first n qubits in circuit (without swaps)"""
        if n == 0:
            return circuit
        n -= 1
        circuit.h(n)
        for qubit in range(n):
            circuit.cp(np.pi/2**(n-qubit), qubit, n)
        qft_rotations(circuit, n)
    
    qft_rotations(circuit, n)
    swap_registers(circuit, n)
    return circuit

上記で作成したqft呼び出し時の引数をcountBitに変更。

In [103]:
# 作成したqftの引数をcountBitに変更
qft_dagger = qft(countBit).to_gate().inverse()
qft_dagger.label = "QFT†"

In [104]:
# 量子回路の作成
t = countBit   # カウントする量子ビットの数
n = 4   # 探索する量子ビットの数
qc = QuantumCircuit(n+t, t) # 古典ビットとn+t量子ビットの回路

# 全ての量子ビットを |+>に初期化する
for qubit in range(t+n):
    qc.h(qubit)

# Begin controlled Grover iterations
iterations = 1
for qubit in range(t):
    for i in range(iterations):
        qc.append(cgrit, [qubit] + [*range(t, n+t)])
    iterations *= 2
    
# Do inverse QFT on counting qubits
qc.append(qft_dagger, range(t))

# Measure counting qubits
qc.measure(range(t), range(t))

# Display the circuit
qc.draw()

┌───┐                                                                  »
q_0: ┤ H ├─────■────────────────────────────────────────────────────────────»
     ├───┤     │                                                            »
q_1: ┤ H ├─────┼──────────■──────────■──────────────────────────────────────»
     ├───┤     │          │          │                                      »
q_2: ┤ H ├─────┼──────────┼──────────┼──────────■──────────■──────────■─────»
     ├───┤┌────┴────┐┌────┴────┐┌────┴────┐┌────┴────┐┌────┴────┐┌────┴────┐»
q_3: ┤ H ├┤0        ├┤0        ├┤0        ├┤0        ├┤0        ├┤0        ├»
     ├───┤│         ││         ││         ││         ││         ││         │»
q_4: ┤ H ├┤1        ├┤1        ├┤1        ├┤1        ├┤1        ├┤1        ├»
     ├───┤│  Grover ││  Grover ││  Grover ││  Grover ││  Grover ││  Grover │»
q_5: ┤ H ├┤2        ├┤2        ├┤2        ├┤2        ├┤2        ├┤2        ├»
     ├───┤│         ││         ││         ││         ││         ││         │»
q_6: ┤ H ├┤3        ├┤3        ├┤3        ├┤3        ├┤3        ├┤3        ├»
     └───┘└─────────┘└─────────┘└─────────┘└─────────┘└─────────┘└─────────┘»
c: 3/═══════════════════════════════════════════════════════════════════════»
                                                                            »
«                ┌───────┐┌─┐      
«q_0: ───────────┤0      ├┤M├──────
«                │       │└╥┘┌─┐   
«q_1: ───────────┤1 QFT† ├─╫─┤M├───
«                │       │ ║ └╥┘┌─┐
«q_2: ─────■─────┤2      ├─╫──╫─┤M├
«     ┌────┴────┐└───────┘ ║  ║ └╥┘
«q_3: ┤0        ├──────────╫──╫──╫─
«     │         │          ║  ║  ║ 
«q_4: ┤1        ├──────────╫──╫──╫─
«     │  Grover │          ║  ║  ║ 
«q_5: ┤2        ├──────────╫──╫──╫─
«     │         │          ║  ║  ║ 
«q_6: ┤3        ├──────────╫──╫──╫─
«     └─────────┘          ║  ║  ║ 
«c: 3/═════════════════════╩══╩══╩═
«                          0  1  2

回路を見ると反復回数が増加している。

In [105]:
aer_sim = Aer.get_backend('aer_simulator')
transpiled_qc = transpile(qc, aer_sim)
qobj = assemble(transpiled_qc)
job = aer_sim.run(qobj)
hist = job.result().get_counts()

measured_str = max(hist, key=hist.get)
measured_int = int(measured_str,2)
print("Register Output = %i" % measured_int)
theta = (measured_int/(2**t))*math.pi*2
print("θ = %.5f" % theta)
N = 2**n
M = N * (math.sin(theta/2)**2)
print("解の数 = %.1f" % (N-M))
m = t - 1 # Upper bound: Will be less than this 
err = (math.sqrt(2*M*N) + N/(2**(m+1)))*(2**(-m))
print("Error < %.2f" % err)

Register Output = 6
θ = 4.71239
解の数 = 8.0
Error < 4.50


カウント量子ビットを10にした場合<br><br>
Register Output = 705<br>
θ = 4.32583<br>
解の数 = 5.0<br>
Error < 0.04<br>

解の数が5.0になり、カウント量子ビットが4の場合より正確な値になっている。<br>エラーも0.04まで減少している。

カウント量子ビットを3にした場合<br>

Register Output = 6<br>
θ = 4.71239<br>
解の数 = 8.0<br>
Error < 4.50

解の数が8.0になり、実際の解の数との差が大きくなっている。<br>
エラーも4.50まで増加している。

A.上記の結果より、カウント量子ビットの数を増加させると精度が高くなり、
減少させると精度が低くなることが分かる。